# Project M2 - Cell Type Classification with Morphology features
### The objective of this project is to classify two cell types (spiny/aspiny) according to their Morphology features using both logistic regression and neural network.
#### The data set is downloaded from the __[Allen Institute data base](http://alleninstitute.github.io/AllenSDK/_static/examples/nb/cell_types.html#Computing-Electrophysiology-Features)__ and is already saved in the file "MorphFeatures.csv". Train the classifiers similarly to Project M1 using only morphology featrues. Then try to combine them with electrophysiology features to see how that would change the accuracy.

## Getting start with the Allen's data set
Use python library Pandas to read the csv file. The data set is now stored in Pandas dataframe.

In [2]:
import numpy as np
import pandas as pd

df = pd.read_csv("MorphFeatures.csv",index_col=0)
print(df.shape)
df.head(5) #show some lines on the data from the first record.

(670, 31)


,average_bifurcation_angle_local,average_bifurcation_angle_remote,average_contraction,average_diameter,average_fragmentation,average_parent_daughter_ratio,hausdorff_dimension,id,max_branch_order,max_euclidean_distance,...,scale_factor_y,scale_factor_z,soma_surface,specimen_id,superseded,tags,total_length,total_surface,total_volume,dendrite_type
0,82.727781,NaN,0.864267,0.345092,20.723077,0.964510,NaN,491119743,6.0,99.779724,...,0.1144,0.28,435.740270,478107198,False,3D Neuron Reconstruction morphology,1666.082926,1803.875644,167.343086,aspiny
1,82.506680,NaN,0.903890,0.634047,105.277778,0.862183,NaN,546781359,3.0,432.383110,...,0.1144,0.28,1446.587725,502367941,False,3D Neuron Reconstruction morphology,2277.259374,4543.139073,921.571895,spiny
2,77.536678,NaN,0.863104,0.417929,73.666667,0.926633,NaN,537042261,6.0,373.630444,...,0.1144,0.28,287.118123,515771244,False,3D Neuron Reconstruction morphology,3589.339062,4704.910407,582.285423,spiny
3,76.583222,NaN,0.900537,0.400396,95.979167,0.942049,NaN,689123605,11.0,943.382549,...,0.1144,0.28,180.994813,561435279,False,3D Neuron Reconstruction morphology,5416.228778,6814.933290,740.722806,spiny
4,72.019250,NaN,0.873518,0.227626,47.535714,1.000000,NaN,657879305,5.0,186.218009,...,0.1144,0.28,55.055236,591268268,False,3D Neuron Reconstruction morphology,1659.465869,1185.773462,69.144146,aspiny


The cell type is determined by the dendrite type in the last column of the data set. Ignore the samples of minority type called "sparsely spiny".

In [3]:
df.dropna(axis=1,inplace=True) # Drop columns with Nan values
df = df.drop_duplicates(subset=['specimen_id']) # drop duplicated of specimen_id

# Get rid of sparsely spiny cells
df = df[df.dendrite_type!='sparsely spiny'] #keep all the data that 'dendrite_type' is not 'sparsely spiny'
print(df.shape)
print(df.columns)
df.head(5)

(619, 29)
Index(['average_bifurcation_angle_local', 'average_contraction',
       'average_diameter', 'average_fragmentation',
       'average_parent_daughter_ratio', 'id', 'max_branch_order',
       'max_euclidean_distance', 'max_path_distance',
       'neuron_reconstruction_type', 'number_bifurcations', 'number_branches',
       'number_nodes', 'number_stems', 'number_tips', 'overall_depth',
       'overall_height', 'overall_width', 'scale_factor_x', 'scale_factor_y',
       'scale_factor_z', 'soma_surface', 'specimen_id', 'superseded', 'tags',
       'total_length', 'total_surface', 'total_volume', 'dendrite_type'],
      dtype='object')


,average_bifurcation_angle_local,average_contraction,average_diameter,average_fragmentation,average_parent_daughter_ratio,id,max_branch_order,max_euclidean_distance,max_path_distance,neuron_reconstruction_type,...,scale_factor_y,scale_factor_z,soma_surface,specimen_id,superseded,tags,total_length,total_surface,total_volume,dendrite_type
0,82.727781,0.864267,0.345092,20.723077,0.964510,491119743,6.0,99.779724,126.593790,dendrite-only,...,0.1144,0.28,435.740270,478107198,False,3D Neuron Reconstruction morphology,1666.082926,1803.875644,167.343086,aspiny
1,82.506680,0.903890,0.634047,105.277778,0.862183,546781359,3.0,432.383110,496.831994,dendrite-only,...,0.1144,0.28,1446.587725,502367941,False,3D Neuron Reconstruction morphology,2277.259374,4543.139073,921.571895,spiny
2,77.536678,0.863104,0.417929,73.666667,0.926633,537042261,6.0,373.630444,436.958952,dendrite-only,...,0.1144,0.28,287.118123,515771244,False,3D Neuron Reconstruction morphology,3589.339062,4704.910407,582.285423,spiny
3,76.583222,0.900537,0.400396,95.979167,0.942049,689123605,11.0,943.382549,989.448317,full,...,0.1144,0.28,180.994813,561435279,False,3D Neuron Reconstruction morphology,5416.228778,6814.933290,740.722806,spiny
4,72.019250,0.873518,0.227626,47.535714,1.000000,657879305,5.0,186.218009,221.639502,full,...,0.1144,0.28,55.055236,591268268,False,3D Neuron Reconstruction morphology,1659.465869,1185.773462,69.144146,aspiny


## Feature Engineering

In [4]:
import seaborn as sns
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

df_full = df.drop(columns=['id','neuron_reconstruction_type','scale_factor_x', 'scale_factor_y','scale_factor_z','specimen_id','superseded','tags'])
print(df_full.shape)
df_full.head()

(619, 21)


,average_bifurcation_angle_local,average_contraction,average_diameter,average_fragmentation,average_parent_daughter_ratio,max_branch_order,max_euclidean_distance,max_path_distance,number_bifurcations,number_branches,...,number_stems,number_tips,overall_depth,overall_height,overall_width,soma_surface,total_length,total_surface,total_volume,dendrite_type
0,82.727781,0.864267,0.345092,20.723077,0.964510,6.0,99.779724,126.593790,33,73,...,7,40,51.4886,140.506829,136.267522,435.740270,1666.082926,1803.875644,167.343086,aspiny
1,82.506680,0.903890,0.634047,105.277778,0.862183,3.0,432.383110,496.831994,9,23,...,5,14,92.6671,566.701220,370.170045,1446.587725,2277.259374,4543.139073,921.571895,spiny
2,77.536678,0.863104,0.417929,73.666667,0.926633,6.0,373.630444,436.958952,21,46,...,4,25,65.4696,425.897625,381.015114,287.118123,3589.339062,4704.910407,582.285423,spiny
3,76.583222,0.900537,0.400396,95.979167,0.942049,11.0,943.382549,989.448317,24,52,...,4,28,99.9139,1217.694976,524.550156,180.994813,5416.228778,6814.933290,740.722806,spiny
4,72.019250,0.873518,0.227626,47.535714,1.000000,5.0,186.218009,221.639502,14,32,...,4,18,54.3718,172.075941,261.459057,55.055236,1659.465869,1185.773462,69.144146,aspiny


In [5]:
X = abs(df_full.iloc[:,:-1]) # Need to take absolute value for SelectKBest to work
Y = df_full.iloc[:,-1]       # iloc Purely integer-location based indexing for selection by position.

In [6]:
bestfeatures = SelectKBest(score_func=chi2)
fit = bestfeatures.fit(X,Y)

dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(df_full.columns)

featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns

print(featureScores.nlargest(20,'Score'))  #print 20 best features

                              Specs          Score
18                    total_surface  539939.690284
17                     total_length  264251.289979
10                     number_nodes  220597.718099
19                     total_volume  115805.409932
7                 max_path_distance   30418.233712
6            max_euclidean_distance   27774.577274
14                   overall_height   23066.856685
9                   number_branches    2410.349432
15                    overall_width    2214.133645
13                    overall_depth    1994.393610
12                      number_tips    1213.162125
8               number_bifurcations    1201.211153
16                     soma_surface     905.144986
5                  max_branch_order     556.265637
11                     number_stems      64.412472
3             average_fragmentation      55.236331
0   average_bifurcation_angle_local      16.120053
2                  average_diameter       0.834203
4     average_parent_daughter_r

In [7]:
nbest = 3 # Using 3 best features for example
pick_feats = list(featureScores.nlargest(nbest,'Score').Specs) # make a list of the n best features
pick_feats.append('dendrite_type') # add dendrite_type to the list

df_small = df[pick_feats] # Make a new DataFrame with our selected features
df_small.head(5)

,total_surface,total_length,number_nodes,dendrite_type
0,1803.875644,1666.082926,1470,aspiny
1,4543.139073,2277.259374,2011,spiny
2,4704.910407,3589.339062,3137,spiny
3,6814.933290,5416.228778,4652,spiny
4,1185.773462,1659.465869,1406,aspiny


Make pair plot to visualize the features distribution if you want.
#### Now you have defined the training data set and the class labels. Next train the logistic regression classifier and the neural network like in the two examples and compare the performance of these two methods.
## Example for logistic regression

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder, LabelBinarizer, scale
from sklearn.model_selection import train_test_split, KFold, cross_val_score

seed = 12345 # random seed

X = df_small.values[:,:-1]
Y = df_small.values[:,-1]
lb = LabelBinarizer()   # Binarize labels in a one-vs-all fashion
y = lb.fit_transform(Y).ravel() # convert values in Y into binary labels

Split the dataset into two groups for training and testing.

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=seed)  # random pick 25% for testing
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(464, 3) (155, 3) (464,) (155,)


Train and test in different groups.

In [10]:
# Create logistic regression classifier
log_reg = LogisticRegression(penalty='l2',solver='liblinear')

log_reg.fit(X_train,y_train)

y_pred = log_reg.predict(X_test)  # Predict class labels for samples in X_test.

print("Model accuracy:", accuracy_score(y_test,y_pred))     # Calculate the accuracy comparing the predicted labels with the ground truth
print("Confusion matrix", confusion_matrix(y_test, y_pred)) # Calculate the confusion matrix

Model accuracy: 0.7548387096774194
Confusion matrix [[53 14]
 [24 64]]


5-fold cross-validation.

In [11]:
kfold = KFold(n_splits=5, shuffle=True, random_state=seed)
results = cross_val_score(log_reg, X, y, cv=kfold)
print("Model Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Model Results: 74.79% (4.15%)


## Example for random forest
Random forest is one of the algorithms using emsemble method. A random forest is a meta estimator that fits a number of decision tree classifiers on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting.

In [12]:
from sklearn.ensemble import RandomForestClassifier

# Create random forest classifier
rand_for = RandomForestClassifier(n_estimators=10, criterion='gini', random_state=seed)
# n_estimators is the number of trees in the forest
# criterion it the function to measure the quality of a split.'gini' for the Gini impurity and 'entropy' for the information gain.

Train and test the random forest classifier.

In [13]:
rand_for.fit(X_train,y_train)
y_pred = rand_for.predict(X_test)

print("Model accuracy:", accuracy_score(y_test,y_pred))     # Calculate the accuracy comparing the predicted labels with the ground truth
print("Confusion matrix", confusion_matrix(y_test, y_pred)) # Calculate the confusion matrix

Model accuracy: 0.7225806451612903
Confusion matrix [[53 14]
 [29 59]]


5 fold cross-validation.

In [14]:
kfold = KFold(n_splits=5, shuffle=True, random_state=seed)
results = cross_val_score(rand_for, X, y, cv=kfold)
print("Model Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Model Results: 73.67% (2.59%)


## Other emsemble methods
Check out an example __[Boosting, Bagging, and Stacking — Ensemble Methods with sklearn and mlens](https://medium.com/@rrfd/boosting-bagging-and-stacking-ensemble-methods-with-sklearn-and-mlens-a455c0c982de)__ applied to Housing.
### Bagging
Bagging, is shorthand for the combination of bootstrapping and aggregating. Bootstrapping is a method to help decrease the variance of the classifier and reduce overfitting, by resampling data from the training set with the same cardinality as the original set. The model created should be less overfitted than a single individual model.  
A high variance for a model is not good, suggesting its performance is sensitive to the training data provided. So, even if more the training data is provided, the model may still perform poorly. And, may not even reduce the variance of our model.  
Bagging is an effective method when you have limited data, and by using samples you’re able to get an estimate by aggregating the scores over many samples.

A Bagging classifier is an ensemble meta-estimator that fits base classifiers each on random subsets of the original dataset and then aggregate their individual predictions (either by voting or by averaging) to form a final prediction. Following example shows an emsemble of multiple classification algorithms.

In [15]:
from sklearn.ensemble import BaggingClassifier, ExtraTreesClassifier, RandomForestClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import SVC
import warnings
warnings.filterwarnings('ignore')

# Create classifiers
rf = RandomForestClassifier(n_estimators=10)
et = ExtraTreesClassifier()
knn = KNeighborsClassifier()
svc = SVC()
rg = RidgeClassifier()

# Put them in a list
clf_array = [rf, et, knn, svc, rg]

Apply bagging on each classification method.

In [16]:
for clf in clf_array:
    vanilla_scores = cross_val_score(clf, X, y, cv=kfold, n_jobs=-1)
    bagging_clf = BaggingClassifier(clf, max_samples=0.25, max_features=1.0, random_state=seed)
    # max_samples: The proportion of samples to draw from X to train each base estimator.
    # max_features: The proportion of features to draw from X to train each base estimator.
    bagging_scores = cross_val_score(bagging_clf, X, y, cv=10, n_jobs=-1)
    
    print("Mean of: {1:.3f}, std: (+/-) {2:.3f} [{0}]".format(clf.__class__.__name__, vanilla_scores.mean()*100, vanilla_scores.std()*100))
    print("Mean of: {1:.3f}, std: (+/-) {2:.3f} [Bagging {0}]\n".format(clf.__class__.__name__, bagging_scores.mean()*100, bagging_scores.std()*100))

Mean of: 71.887, std: (+/-) 4.841 [RandomForestClassifier]
Mean of: 75.275, std: (+/-) 4.204 [Bagging RandomForestClassifier]

Mean of: 71.728, std: (+/-) 2.225 [ExtraTreesClassifier]
Mean of: 76.571, std: (+/-) 4.693 [Bagging ExtraTreesClassifier]

Mean of: 72.864, std: (+/-) 3.842 [KNeighborsClassifier]
Mean of: 76.076, std: (+/-) 5.511 [Bagging KNeighborsClassifier]

Mean of: 55.733, std: (+/-) 1.252 [SVC]
Mean of: 55.736, std: (+/-) 0.253 [Bagging SVC]

Mean of: 72.370, std: (+/-) 5.762 [RidgeClassifier]
Mean of: 73.019, std: (+/-) 4.427 [Bagging RidgeClassifier]



Use VotingClassifier to combine different machine learning classifiers, and perform a vote on what the predicted class labels are.

In [17]:
# Set up voting
eclf = VotingClassifier(estimators=[('Random Forests',rf), ('Extra Trees',et), ('KNeighbors',knn), ('SVC',svc), ('Ridge Classifier',rg)],
                        voting='hard')

for clf, label in zip([rf, et, knn, svc, rg, eclf], ['Random Forest','Extra Trees','KNeighbors','SVC','Ridge Classifier','Ensemble']):
    scores = cross_val_score(clf, X, y, cv=10, scoring='accuracy')
    print("Mean: {0:.3f}, std: (+/-) {1:.3f} [{2}]".format(scores.mean()*100, scores.std()*100, label))

Mean: 72.194, std: (+/-) 5.800 [Random Forest]
Mean: 71.723, std: (+/-) 4.486 [Extra Trees]
Mean: 73.846, std: (+/-) 4.990 [KNeighbors]
Mean: 55.736, std: (+/-) 0.253 [SVC]
Mean: 72.832, std: (+/-) 3.859 [Ridge Classifier]
Mean: 73.016, std: (+/-) 3.972 [Ensemble]


Apply voting for bagging on different classifiers.

In [18]:
# Set up ensemble voting for bagging
ebclf_array = []

for clf in clf_array:
    ebclf_array.append(BaggingClassifier(clf, max_samples=0.25, max_features=1.0, random_state=seed))

v_eclf = VotingClassifier(estimators=list(zip(['Bagging Random Forest','Bagging Extra Trees','Bagging KNeighbors','Bagging SVC','Bagging Ridge Classifier'],ebclf_array)),  voting='hard')
ebclf_array.append(v_eclf)

for clf, label in zip(ebclf_array, ['Bagging Random Forest', 'Bagging Extra Trees', 'Bagging KNeighbors',
                              'Bagging SVC', 'BaggingRidge Classifier' ,'Bagging Ensemble']):
    scores = cross_val_score(clf, X, y, cv=kfold, scoring='accuracy')
    print("Mean: {0:.3f}, std: (+/-) {1:.3f} [{2}]".format(scores.mean(), scores.std(), label))

Mean: 0.764, std: (+/-) 0.030 [Bagging Random Forest]
Mean: 0.761, std: (+/-) 0.029 [Bagging Extra Trees]
Mean: 0.748, std: (+/-) 0.032 [Bagging KNeighbors]
Mean: 0.557, std: (+/-) 0.013 [Bagging SVC]
Mean: 0.719, std: (+/-) 0.059 [BaggingRidge Classifier]
Mean: 0.762, std: (+/-) 0.034 [Bagging Ensemble]


### AdaBoost
AdaBoost is an ensemble algorithm for manipulating the training set. An AdaBoost classifier is a meta-estimator that begins by fitting a classifier on the original dataset and then fits additional copies of the classifier on the same dataset but where the weights of incorrectly classified instances are adjusted such that subsequent classifiers focus more on difficult cases.  
In the following example, the emsembled classifiers are decision trees.

In [19]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

ada_boost = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(criterion='gini'), n_estimators=10, random_state=seed)

In [20]:
ada_boost.fit(X_train,y_train)
y_pred = ada_boost.predict(X_test)

print("Model accuracy:", accuracy_score(y_test,y_pred))     # Calculate the accuracy comparing the predicted labels with the ground truth
print("Confusion matrix", confusion_matrix(y_test, y_pred)) # Calculate the confusion matrix

Model accuracy: 0.7290322580645161
Confusion matrix [[49 18]
 [24 64]]


In [21]:
kfold = KFold(n_splits=5, shuffle=True, random_state=seed)
results = cross_val_score(ada_boost, X, y, cv=kfold)
print("Model Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Model Results: 69.63% (3.80%)


### Select more featrues and apply all the algorithms shown above also including artificial neural network. Try to understand the emsemble methods and comment on the results.
### After you finish training on the morphology featrues, try to combine with the electrophysiology features and see how it will affect the results.
You can find the corresponding cell in the electrophysiology features dataset according to the specimen_id. Use following lines to combine the two data sets.

In [22]:
df_ef = pd.read_csv("ElecPhyFeatures.csv",index_col=0)       # Read electrophysiology data
print(df.shape)
print(df_ef.shape)

df_cb = pd.merge(df,df_ef,how='inner',left_on='specimen_id', right_on='specimen_id',suffixes=('_mp','')) # Combine two data frames
print(df_cb.shape)

(619, 29)
(2333, 57)
(619, 85)


You need to pick feature columns from the combined dataset for training.

In [23]:
df_cb = df_cb.drop(columns='dendrite_type_mp') # dendrite_type was duplicated while merging two dataframes, drop out one of them.
print(df_cb.shape)
print(df_cb.columns)
df_cb.head(5)

(619, 84)
Index(['average_bifurcation_angle_local', 'average_contraction',
       'average_diameter', 'average_fragmentation',
       'average_parent_daughter_ratio', 'id_mp', 'max_branch_order',
       'max_euclidean_distance', 'max_path_distance',
       'neuron_reconstruction_type', 'number_bifurcations', 'number_branches',
       'number_nodes', 'number_stems', 'number_tips', 'overall_depth',
       'overall_height', 'overall_width', 'scale_factor_x', 'scale_factor_y',
       'scale_factor_z', 'soma_surface', 'specimen_id', 'superseded', 'tags',
       'total_length', 'total_surface', 'total_volume', 'adaptation',
       'avg_isi', 'electrode_0_pa', 'f_i_curve_slope',
       'fast_trough_t_long_square', 'fast_trough_t_ramp',
       'fast_trough_t_short_square', 'fast_trough_v_long_square',
       'fast_trough_v_ramp', 'fast_trough_v_short_square', 'has_burst',
       'has_delay', 'has_pause', 'id', 'input_resistance_mohm', 'latency',
       'peak_t_long_square', 'peak_t_ramp', 'pea

,average_bifurcation_angle_local,average_contraction,average_diameter,average_fragmentation,average_parent_daughter_ratio,id_mp,max_branch_order,max_euclidean_distance,max_path_distance,neuron_reconstruction_type,...,trough_t_short_square,trough_v_long_square,trough_v_ramp,trough_v_short_square,upstroke_downstroke_ratio_long_square,upstroke_downstroke_ratio_ramp,upstroke_downstroke_ratio_short_square,vm_for_sag,vrest,dendrite_type
0,82.727781,0.864267,0.345092,20.723077,0.964510,491119743,6.0,99.779724,126.593790,dendrite-only,...,1.047640,-59.812500,-59.437504,-66.718750,2.625872,2.969025,1.909216,-77.718758,-63.952812,aspiny
1,82.506680,0.903890,0.634047,105.277778,0.862183,546781359,3.0,432.383110,496.831994,dendrite-only,...,1.114696,-58.125004,-57.864586,-66.443753,4.322536,4.272167,3.659369,-98.937508,-66.457214,spiny
2,77.536678,0.863104,0.417929,73.666667,0.926633,537042261,6.0,373.630444,436.958952,dendrite-only,...,1.236256,-56.000000,-57.979170,-69.356253,5.076478,5.069481,3.992479,-85.312500,-68.378876,spiny
3,76.583222,0.900537,0.400396,95.979167,0.942049,689123605,11.0,943.382549,989.448317,full,...,1.096650,-55.031250,-55.989587,-63.726564,3.445772,3.264711,3.118708,-83.625000,-62.545727,spiny
4,72.019250,0.873518,0.227626,47.535714,1.000000,657879305,5.0,186.218009,221.639502,full,...,1.377193,-60.656254,-54.562500,-72.192708,1.255817,1.051984,1.371654,-85.093750,-71.524811,aspiny
